In [148]:
import pandas as pd
# Importing the data
DATA_PATH_GENRE = 'https://raw.githubusercontent.com/Spotify-Song-Predict-DS18/Song-Predict/main/data/data_w_genres.csv'
DATA_PATH_GENRE_2 = 'https://raw.githubusercontent.com/Spotify-Song-Predict-DS18/Song-Predict/main/data/data_by_genres.csv'
DATA_PATH_ARTIST = 'https://raw.githubusercontent.com/Spotify-Song-Predict-DS18/Song-Predict/main/data/data_by_artist.csv'
DATA_PATH = 'https://raw.githubusercontent.com/Spotify-Song-Predict-DS18/Song-Predict/main/data/data.csv'
genres = pd.read_csv(DATA_PATH_GENRE)
genre2 = pd.read_csv(DATA_PATH_GENRE_2)
artist = pd.read_csv(DATA_PATH_ARTIST)
control = pd.read_csv(DATA_PATH)

In [149]:
#CLEANING DATA: REMOVING '' and [] SYMBOLS

list_of_genres = []
for i in genres['genres']:
    i = i[1:-1].replace("'", "")
    list_of_genres.append(i)
    
genres.drop(['genres'], axis=1, inplace=True)
genres = genres.assign(New_genres=list_of_genres)
genres = genres.rename(columns={'New_genres':'genre', 'artists':'artist'})

#removing empty values from 'genre' column
genres = genres[genres['genre'] != '']

In [150]:
#CLEANING "ARTISTS": REMOVING [] and ''
list_of_artists  = []
for k in control['artists']:
    k = k[1:-1].replace("'", "")
    list_of_artists.append(k)
list_of_artists   
control.drop(['artists'], axis=1, inplace=True)
control = control.assign(New_artists=list_of_artists)
control = control.rename(columns={'New_artists':'artist'})

In [151]:
# Merging 
result = pd.merge(control, genres[['artist', 'genre']], how='left', on='artist')
result = result[result['genre'].isnull() == False]

In [152]:
# Transforming my dataset
# TODO - Dates need to be coherent
# TODO - might have to make it into a function. 

result['release_date'] = pd.to_datetime(control['release_date'])

# Model crashes so taking out all music before the beginning 2020
# mask = result['release_date'] >= '2020-1-1'
# result = result[mask]

In [163]:
# Creating a "user column" where it assigns each song to a "user"
import numpy as np

# Applying random "users" to the music
# Perfect for predictions.
result['user'] = np.random.randint(0, 200, result.shape[0])

# Checking if the function works
result.head(10)

,acousticness,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,...,name,popularity,release_date,speechiness,tempo,valence,year,artist,genre,user
3,0.995,0.781,180760,0.130,0,6M94FkXd15sOAOQYRnWPN8,0.887000,1,0.1110,-14.734,...,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.720,1928,Francisco Canaro,"tango, vintage tango",73
19,0.992,0.763,173707,0.132,0,6Ukl7n0q3Cjd0Og8uBmVeP,0.069300,4,0.1120,-13.002,...,Tendrás Que Llorar Conmigo - Instrumental (Rem...,0,1928-09-17,0.0886,111.679,0.832,1928,Francisco Canaro,"tango, vintage tango",146
21,0.990,0.833,176000,0.132,0,6VGWQU6XIXshFT288CoLrk,0.000568,1,0.0957,-11.131,...,Quisiste Cachar un Gil - Instrumental (Remaste...,0,1928-09-20,0.2950,121.779,0.568,1928,Francisco Canaro,"tango, vintage tango",101
23,0.990,0.811,169720,0.175,0,6VqUdRHLlFMXmN4plGWSMA,0.841000,9,0.1300,-13.639,...,La Recova - Instrumental (Remasterizado),0,1928-09-16,0.0712,115.212,0.933,1928,Francisco Canaro,"tango, vintage tango",115
31,0.995,0.531,159107,0.124,0,6XoyWGdCJwFaJV1Pnmphwr,0.016800,2,0.1180,-23.243,...,Por una Mujer - Remasterizado,0,1928-05-14,0.0711,101.902,0.555,1928,Ignacio Corsini,"tango, vintage tango",158
34,0.983,0.668,151840,0.261,0,6ZSXaYQBOPDDmKeqaOfBa2,0.023200,7,0.1220,-16.910,...,Abuelita - Remasterizado,0,1928-05-14,0.1010,120.553,0.702,1928,Ignacio Corsini,"tango, vintage tango",41
35,0.989,0.769,140267,0.442,0,6ZWi1fuonJCUlt4p6o9Uzs,0.000001,2,0.1670,-12.697,...,La Noce À Rebecca,0,1928-01-01,0.2730,132.125,0.934,1928,Perchicot,vintage chanson,143
37,0.994,0.854,200213,0.151,0,6aNewFey3lEHDerGgy3rYN,0.371000,4,0.1590,-11.163,...,Che Bouhier - Instrumental (Remasterizado),0,1928-09-16,0.1370,110.331,0.781,1928,Francisco Canaro,"tango, vintage tango",141
43,0.995,0.729,174720,0.191,0,6hcPbniXegOud4FNv8Efxz,0.455000,4,0.1030,-15.313,...,Timbero - Instrumental (Remasterizado),0,1928-09-17,0.1080,112.565,0.902,1928,Francisco Canaro,"tango, vintage tango",141
46,0.985,0.810,169160,0.300,0,6iInEwECoC9ZX8uPDRjyft,0.000885,4,0.1300,-9.702,...,Mariposa de Cabaret - Instrumental (Remasteriz...,0,1928-09-25,0.2890,115.648,0.806,1928,Francisco Canaro,"tango, vintage tango",37


In [164]:
# TODO - Build a recommender model with machine learning. 
from surprise import Reader, Dataset, SVD 
from surprise.model_selection.validation import cross_validate

# Intantiating my reader and my data
reader = Reader(rating_scale=(0,100))
data = Dataset.load_from_df(result[['user', 'id', 'popularity']],
                           reader)
# Intatntiating my SVD 
svd = SVD()

In [165]:
# Running a 5-fold cross-validation 
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5,
              verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    18.6509 18.6566 18.6121 18.6578 18.7377 18.6630 0.0409  
MAE (testset)     15.1409 15.1305 15.0744 15.1358 15.2138 15.1391 0.0444  
Fit time          5.26    5.43    5.30    5.25    5.28    5.30    0.07    
Test time         0.09    0.10    0.23    0.09    0.10    0.12    0.05    


{'test_rmse': array([18.65087962, 18.65664585, 18.61214626, 18.65784434, 18.7376646 ]),
 'test_mae': array([15.14086969, 15.13051729, 15.07437847, 15.1357948 , 15.21379387]),
 'fit_time': (5.256954193115234,
  5.429321765899658,
  5.299219846725464,
  5.250789403915405,
  5.279865741729736),
 'test_time': (0.09474635124206543,
  0.09879541397094727,
  0.22539925575256348,
  0.09477877616882324,
  0.10272502899169922)}

In [166]:
# Retraining the model using the entire dataset
trainset = data.build_full_trainset()
svd.fit(trainset)

In [167]:
# Testing a prediction! 
user = '10'
song2 = '78qd8dvwea0Gosb6Fe6j3k'

# Time to test the prediction
pred = svd.predict(user, song2, r_ui=4, verbose=True)

user: 10         item: 78qd8dvwea0Gosb6Fe6j3k r_ui = 4.00   est = 36.92   {'was_impossible': False}


In [168]:
# Model has been trained, time to use for prediction.
titles = result.copy()

titles['Estimate_Score'] = titles['id'].apply(lambda x: svd.predict(65, x).est)

In [169]:
titles = titles.sort_values(by=['Estimate_Score'], ascending=False)
titles.head(5)

,acousticness,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,...,popularity,release_date,speechiness,tempo,valence,year,artist,genre,user,Estimate_Score
126123,0.02220,0.781,180221,0.762,0,0ono6UCNVZ1XqOm6j78Blu,0.000000,10,0.1210,-5.188,...,79,2020-02-21,0.0626,110.042,0.860,2020,BTS,"k-pop, k-pop boy group",65,78.787185
97744,0.13500,0.772,210261,0.634,1,7hav1MHURQf5SAoPk0egqh,0.000000,10,0.4320,-6.101,...,78,2020-03-06,0.2120,160.052,0.671,2020,Lil Uzi Vert,"melodic rap, philly rap, rap",65,77.754125
87238,0.00454,0.622,219720,0.469,0,6FB3v4YcR57y4tXFcdxI1E,0.000002,3,0.0335,-6.798,...,77,2012-10-22,0.0363,77.019,0.679,2012,Taylor Swift,"dance pop, pop, post-teen pop",65,76.862053
82944,0.01300,0.564,146250,0.932,0,78lgmZwycJ3nzsdgmPPGNx,0.169000,11,0.3490,-10.068,...,76,1970-01-01,0.0554,112.937,0.619,1970,Led Zeppelin,"album rock, classic rock, hard rock, rock",65,75.956026
86554,0.01930,0.556,282160,0.913,0,10Nmj3JCNoMeBQ87uw5j8k,0.000009,0,0.3460,-2.360,...,76,2006-05-09,0.0437,96.184,0.730,2006,Red Hot Chili Peppers,"alternative rock, funk metal, funk rock, perma...",65,75.880840


In [170]:
def recommend(userid, df):
    '''A function that will take a userid and predict music based their listen history
    With this database, will only take ids ranging 0-200.
    
    Returns a new column called "Estimate Score" which represents the chance that the
    use will like the music
    
    Score will not be high on average, since users are randomly generated.'''
    
    # First passing through a if statment to make sure 
    # The ID matches the requirements. 
    if userid > 200:
        return "ERROR: User ID too high, must be between 0-200"
    elif userid < 0:
        return "ERROR: User ID too low, must be between 0-200"
    else:
        print("Generating music recommendation...")
    
    # Intantiating my reader and my data
    reader = Reader(rating_scale=(0,100))
    data = Dataset.load_from_df(result[['user', 'id', 'popularity']],
                           reader)
    # Intatntiating my SVD 
    svd = SVD()
    
    # Running a 5-fold cross-validation 
    cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5,
                  verbose=True)
    
    # Retraining the model using the entire dataset
    trainset = data.build_full_trainset()
    svd.fit(trainset)
    
    # Model has been trained, time to use for prediction.
    titles = result.copy()

    titles['Estimate_Score'] = titles['id'].apply(lambda x: svd.predict(userid, x).est)
    
    # Creating a mask that does not include songs the user already has. 
    mask = titles['user'] != userid
    titles = titles[mask]
    
    # Now returning the top 5 songs for that user.
    titles = titles.sort_values(by=['Estimate_Score'], ascending=False)
    return titles.head(5)

In [171]:
# Insert User id (int) and Dataframe into function
userid = 35
recommend(userid, result)

Generating music recommendation...
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    18.7282 18.5569 18.7786 18.5758 18.6777 18.6634 0.0857  
MAE (testset)     15.1559 15.0691 15.2706 15.0669 15.1378 15.1400 0.0744  
Fit time          5.81    5.75    5.59    5.70    5.65    5.70    0.07    
Test time         0.11    0.10    0.25    0.14    0.11    0.14    0.05    


,acousticness,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,...,popularity,release_date,speechiness,tempo,valence,year,artist,genre,user,Estimate_Score
97669,0.153,0.477,202205,0.682,0,6b2RcmUt1g9N9mQ3CbjX2Y,0.00000,1,0.0763,-4.931,...,84,2019-07-19,0.0925,110.567,0.345,2019,Sam Smith,"pop, uk pop",13,47.634002
88087,0.996,0.397,156587,0.229,0,2626aAS6eQgc6CvdLVnoOw,0.83700,6,0.3790,-15.284,...,0,1923-07-09,0.0365,80.086,0.424,1923,Francisco Canaro,"tango, vintage tango",54,47.626142
154260,0.994,0.373,190267,0.422,0,3HXjCpbbxoBVmG4k84tVRj,0.80400,5,0.1230,-11.638,...,0,1948-01-01,0.0930,67.819,0.583,1948,Rashîd al-Qundarjî,middle eastern traditional,56,46.388232
55861,0.954,0.561,195907,0.431,0,4ZY74T0AlsZdMDnpqrstKL,0.91000,3,0.1460,-14.965,...,0,1940-01-01,0.0374,107.661,0.609,1940,Giorgos Kavouras,rebetiko,44,46.211306
136343,0.989,0.554,198520,0.355,0,1JFtd8Wcr1LL9wNXkcGSZX,0.00828,6,0.1510,-10.287,...,0,1940-01-01,0.0379,102.233,0.849,1940,Roza Eskenazi,rebetiko,54,46.170392
